In [199]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score, mean_squared_error
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [200]:
df = pd.read_csv('INE_data.csv')
df.set_index('Date', inplace=True)
df

,city,pernoctaciones,viajeros,precios_hoteleros,ipc_restaurantes_hoteles
Date,,,,,
2023-11-01,0,4050882.0,727579.0,132.51,115.212
2023-10-01,0,5689397.0,1099410.0,139.92,115.776
2023-09-01,0,6783808.0,1325483.0,143.34,115.524
2023-08-01,0,10715411.0,1831834.0,153.00,115.433
2023-07-01,0,9478111.0,1767765.0,150.80,115.114
...,...,...,...,...,...
2017-05-01,17,8841.0,4085.0,87.63,93.717
2017-04-01,17,13748.0,4940.0,89.66,93.772
2017-03-01,17,8419.0,3931.0,82.57,93.057


In [201]:
def predict_value(df, columna, city):
    df_temp = df.copy()
    X_num = pd.DataFrame(df_temp[columna])
    standarizer = StandardScaler().fit(X_num)
    x_standarized = standarizer.transform(X_num)
    df_temp[columna] = x_standarized
    df_city = df_temp[df_temp['city'] == city].sort_index()
    train_data = df_city[columna]['2017-01-01':'2023-12-01']
    train_data.shape
    # Configurar y entrenar el modelo SARIMA
    sarima_model = SARIMAX(train_data,
                        order=(0, 1, 1),  # p, d, q
                        seasonal_order=(0, 1, 1, 12),  # P, D, Q, s (12 para datos mensuales)
                        enforce_stationarity=False,
                        enforce_invertibility=False)

    # Ajustar el modelo
    sarima_results = sarima_model.fit(disp=False)
    forecast = sarima_results.get_forecast(steps=13)
    forecast_values = forecast.predicted_mean
    values = forecast_values.reset_index()
    values['predicted_mean'] = values['predicted_mean'].apply(lambda x : round(standarizer.inverse_transform([[x]])[0][0], 2))
    return values

def predict_merge(city):
    pern = predict_value(df, 'pernoctaciones',city)
    viajeros = predict_value(df, 'viajeros',city)
    precios = predict_value(df, 'precios_hoteleros',city)
    ipc = predict_value(df,'ipc_restaurantes_hoteles',city)
    
    df_merged = pd.merge(pern, viajeros, on='index')
    df_merged.rename(columns={'predicted_mean_x' : 'pernoctaciones', 'predicted_mean_y' : 'viajeros'}, inplace=True)
    df_merged = pd.merge(df_merged, precios, on='index')
    df_merged = pd.merge(df_merged, ipc, on='index')
    df_merged.rename(columns={'predicted_mean_x' : 'precios_hoteleros', 'predicted_mean_y' : 'ipc_restaurantes_hoteles'}, inplace=True)
    df_merged['city'] = city
    df_merged['index'] = pd.to_datetime(df_merged['index']).dt.strftime('%Y-%m-%d')
    df_merged.set_index('index', inplace=True)
    return df_merged

In [202]:
for i in range(0,18,1):
    df = pd.concat([df, predict_merge(i)], axis=0)

c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sta

In [208]:
df.reset_index(names='Date', inplace=True)

In [213]:
df

,Date,city,pernoctaciones,viajeros,precios_hoteleros,ipc_restaurantes_hoteles
0,2023-11-01,0,4050882.00,727579.00,132.51,115.212
1,2023-10-01,0,5689397.00,1099410.00,139.92,115.776
2,2023-09-01,0,6783808.00,1325483.00,143.34,115.524
3,2023-08-01,0,10715411.00,1831834.00,153.00,115.433
4,2023-07-01,0,9478111.00,1767765.00,150.80,115.114
...,...,...,...,...,...,...
1723,2024-08-01,17,41632.40,15027.94,119.36,120.930
1724,2024-09-01,17,19137.32,8999.42,119.45,121.000
1725,2024-10-01,17,20206.23,8231.86,122.37,120.840
1726,2024-11-01,17,13437.66,6299.76,113.93,120.400


In [214]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [217]:
df.to_csv('pred_INE_data.csv', index=False)